In [38]:
#step1
def original_dic():
    dic = {'DIM':'', 'disclosure':'', 'abstract':'', 'general-chemistry':'', 'file-reference-id':'', 'case_code':'', 
           'abst-solution':'', 'title':'', 'tech-problem':'', 'background-art':'', 'description-of-drawings':'', 'description':'', 
           'mode-for-invention':'', 'abstract-dtext':'', 'claim-text':'', 'main-use':'', 'description-of-element':'', 
           'figure-drawings':'', 'abst-problem':'', 'advantageous-effects':'', 'tech-solution':'', 'claim':'', 
           'technical-field':'', 'claims':'', 'sequence-list-text':'', 'drawings':''}
    return dic

def original_ary():
    ary = ['DIM', 'claim', 'abstract', 'general-chemistry', 'file-reference-id', 'case_code',
           'abst-solution', 'drawings', 'title', 'claim-text', 'background-art', 'description-of-drawings', 
           'description', 'mode-for-invention', 'abstract-dtext', 'main-use', 'disclosure', 
           'description-of-element', 'tech-problem', 'advantageous-effects', 'abst-problem', 
           'figure-drawings', 'tech-solution', 'claims', 'sequence-list-text', 'technical-field']
    return ary

In [39]:
#step2
def spec(soup):
    spec_dic = {'id':'', 'lang':'', 'country':'', 'file':'', 'package-num':'', 'receive-num':'', 'receive-date':'',
                'dtd-version':'', 'case-code':'', 'filing-date':'', 'application-num':'', 'ipc-mark':'', 'loc-mark':'',
                'office-num':'', 'status':''}
    if soup.select('specification'):
        spec = soup.select('specification')[0]
        for att in spec.attrs:
            spec_dic[att.encode('utf-8')] = spec[att].encode('utf-8')
            spec_dic['_id'] = spec['application-num']
    elif soup.select('invention-specification'):
        spec = soup.select('invention-specification')[0]
        for att in spec.attrs:
            spec_dic[att.encode('utf-8')] = spec[att].encode('utf-8')
            spec_dic['_id'] = spec['application-num']
    elif soup.select('design-specification'):
        spec = soup.select('design-specification')[0]
        for att in spec.attrs:
            spec_dic[att.encode('utf-8')] = spec[att].encode('utf-8')
            spec_dic['_id'] = spec['application-num']
    elif soup.select('utility-model-specification'):
        spec = soup.select('utility-model-specification')[0]
        for att in spec.attrs:
            spec_dic[att.encode('utf-8')] = spec[att].encode('utf-8')
            spec_dic['_id'] = spec['application-num']
    return spec_dic

#     以下code比較精簡但效率不好
#     ary_spec = ['specification','invention-specification','design-specification','utility-model-specification']
#     for i in range(0,3):
#         if soup.select(ary_spec[i]):
#             spec = soup.select(ary_spec[i])[0]
#             for att in spec.attrs:
#                 spec_dic[att.encode('utf-8')] = spec[att].encode('utf-8')
#                 spec_dic['_id'] = spec['application-num']
#     return spec_dic

In [8]:
#step3
路徑方法
import os, sys
fff = open('E:/patent_data/.txt','w')
path = "E:/patent_data/41" #所有擋案的路徑

PatentPubXML_list = os.listdir( path )
for PatentPubXML_040008 in PatentPubXML_list:
    DMI_list = os.listdir(path +'\\'+ PatentPubXML_040008)  #DMI_list = ['4008M448872', '4008M448873',.....]
    try:
        for DMI in DMI_list:  #DMI = #4008M448872
            xml_list = os.listdir(path +'\\'+ PatentPubXML_040008 + '\\' + DMI)
            for xml in xml_list:
                if os.path.splitext(xml)[-1] == '.xml':  #xml = 101217907-A0305.xml
                    route = path + '/' + PatentPubXML_040008 + '/' + DMI + '/' + xml
                    final_route = route +','+ DMI[4]
                    #print final_route

                    fff.write(final_route + '\n')
    except WindowsError:
        pass
    except:
        print 'other except!!!'

fff.close()

In [42]:
#step4 把換行符號拿掉
#建立read_file方法，參數是xml_route檔案路徑
from bs4 import BeautifulSoup as bs
def read_file(path_and_dim):  #放1279618.xml, D
    dic = original_dic()   #把dic清空
    ary = original_ary()   
    route = path_and_dim.split(',')[0]
    f = open(route,'r')
    soup = bs(f.read())
    dic.update({'DIM':path_and_dim.split(',')[1].encode('utf-8')})
    #dic['DIM'] = path_and_dim.split(',')[1]
    for i in range(0,26):
        if soup.select(ary[i]):
            ss =''
            for a in soup.select(ary[i]):
                ss += a.text.strip().encode('utf-8')
            dic[ary[i]] = ss.replace('\n','')
    if len(spec(soup)) != 0:
        dic.update(spec(soup))
    else:
        print 'No spec:' + path_and_dim
    
    return dic

In [52]:
# dic = read_file('E:/patent_data/40/PatentPubXML_040025/4025I407630/096137619-A0305.xml,I')
# print len(dic)

In [51]:
#for testing
# path_file = open('E:/patent_data/testpath.txt','r')
# #count = 0

# ff = path_file.read().strip()
# #print ff
# doc = read_file(ff)
# #rint len(doc)
# for key in doc:
#     print key,doc[key]
#     #rint '='*100
    
# #rint doc
# #count += 1
    
# #print count  #成功進筆數
# path_file.close()

In [53]:
# from elasticsearch import Elasticsearch
# import elasticsearch
# es = Elasticsearch('10.120.30.10:9200')
# print es
# body={"bingo":"cool"}
# #es.index(index="book1", doc_type="desk1", id=doc['_id'], body=doc, request_timeout=0.2, ignore=400)
# try:
#     es.index(index="book1", doc_type="desk1", id=doc['_id'], body=doc, request_timeout=0.1, ignore=400)
#     count += 1
# except elasticsearch.exceptions.ConnectionError:
#     print 'timeout:' + path_and_dim

In [51]:
#新增資料
from elasticsearch import Elasticsearch
import elasticsearch
es = Elasticsearch('10.120.26.16:9200')

path_file = open('E:/patent_data/path_list_40.txt','r')
count = 0
ff = open('e:/problem.txt','w')
for line in path_file.readlines():
    path_and_dim = line.strip()
    doc = read_file(path_and_dim)
    doc.update({'path_list':'40'})

    try:
        es.index(index="book", doc_type="desk", id=doc['_id'], body=doc, request_timeout=20, ignore=400)
        count += 1
    except elasticsearch.exceptions.ConnectionError:
        print 'timeout:' + path_and_dim
        ff.write('timeout:' + path_and_dim)
        pass
    except:
        print 'error:' + path_and_dim
        ff.write('error:' + path_and_dim)
        pass
    
ff.close()
print '成功筆數:'+str(count)  #成功進筆數
path_file.close()

Traceback (most recent call last):
  File "build\bdist.win-amd64\egg\elasticsearch\connection\http_urllib3.py", line 78, in perform_request
    response = self.pool.urlopen(method, url, body, retries=False, headers=self.headers, **kw)
  File "C:\Users\BigData\Anaconda2\lib\site-packages\urllib3\connectionpool.py", line 609, in urlopen
    _stacktrace=sys.exc_info()[2])
  File "C:\Users\BigData\Anaconda2\lib\site-packages\urllib3\util\retry.py", line 222, in increment
    raise six.reraise(type(error), error, _stacktrace)
  File "C:\Users\BigData\Anaconda2\lib\site-packages\urllib3\connectionpool.py", line 559, in urlopen
    body=body, headers=headers)
  File "C:\Users\BigData\Anaconda2\lib\site-packages\urllib3\connectionpool.py", line 353, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "C:\Users\BigData\Anaconda2\lib\httplib.py", line 1053, in request
    self._send_request(method, url, body, headers)
  File "C:\Users\BigData\Anaconda2\lib\httplib.py", li

error:E:/patent_data/40/PatentPubXML_040028/4028M463005/102210733-A0305.xml,M
error:E:/patent_data/40/PatentPubXML_040028/4028M463008/102210741-A0305.xml,M

Traceback (most recent call last):
  File "build\bdist.win-amd64\egg\elasticsearch\connection\http_urllib3.py", line 78, in perform_request
    response = self.pool.urlopen(method, url, body, retries=False, headers=self.headers, **kw)
  File "C:\Users\BigData\Anaconda2\lib\site-packages\urllib3\connectionpool.py", line 609, in urlopen
    _stacktrace=sys.exc_info()[2])
  File "C:\Users\BigData\Anaconda2\lib\site-packages\urllib3\util\retry.py", line 222, in increment
    raise six.reraise(type(error), error, _stacktrace)
  File "C:\Users\BigData\Anaconda2\lib\site-packages\urllib3\connectionpool.py", line 559, in urlopen
    body=body, headers=headers)
  File "C:\Users\BigData\Anaconda2\lib\site-packages\urllib3\connectionpool.py", line 353, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "C:\Users\BigData\Anaconda2\lib\httplib.py", line 1053, in request
    self._send_request(method, url, body, headers)
  File "C:\Users\BigData\Anaconda2\lib\httplib.py", li


error:E:/patent_data/40/PatentPubXML_040028/4028M463009/102208524-A0305.xml,M

Traceback (most recent call last):
  File "build\bdist.win-amd64\egg\elasticsearch\connection\http_urllib3.py", line 78, in perform_request
    response = self.pool.urlopen(method, url, body, retries=False, headers=self.headers, **kw)
  File "C:\Users\BigData\Anaconda2\lib\site-packages\urllib3\connectionpool.py", line 609, in urlopen
    _stacktrace=sys.exc_info()[2])
  File "C:\Users\BigData\Anaconda2\lib\site-packages\urllib3\util\retry.py", line 222, in increment
    raise six.reraise(type(error), error, _stacktrace)
  File "C:\Users\BigData\Anaconda2\lib\site-packages\urllib3\connectionpool.py", line 559, in urlopen
    body=body, headers=headers)
  File "C:\Users\BigData\Anaconda2\lib\site-packages\urllib3\connectionpool.py", line 353, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "C:\Users\BigData\Anaconda2\lib\httplib.py", line 1053, in request
    self._send_request(method, url, body, headers)
  File "C:\Users\BigData\Anaconda2\lib\httplib.py", li

...


error:E:/patent_data/40/PatentPubXML_040028/4028M463011/102209063-A0305.xml,M

Traceback (most recent call last):
  File "build\bdist.win-amd64\egg\elasticsearch\connection\http_urllib3.py", line 78, in perform_request
    response = self.pool.urlopen(method, url, body, retries=False, headers=self.headers, **kw)
  File "C:\Users\BigData\Anaconda2\lib\site-packages\urllib3\connectionpool.py", line 609, in urlopen
    _stacktrace=sys.exc_info()[2])
  File "C:\Users\BigData\Anaconda2\lib\site-packages\urllib3\util\retry.py", line 222, in increment
    raise six.reraise(type(error), error, _stacktrace)
  File "C:\Users\BigData\Anaconda2\lib\site-packages\urllib3\connectionpool.py", line 559, in urlopen
    body=body, headers=headers)
  File "C:\Users\BigData\Anaconda2\lib\site-packages\urllib3\connectionpool.py", line 353, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "C:\Users\BigData\Anaconda2\lib\httplib.py", line 1053, in request
    self._send_request(method, url, body, headers)
  File "C:\Users\BigData\Anaconda2\lib\httplib.py", li


error:E:/patent_data/40/PatentPubXML_040028/4028M463012/102207874-A0305.xml,M

Traceback (most recent call last):
  File "build\bdist.win-amd64\egg\elasticsearch\connection\http_urllib3.py", line 78, in perform_request
    response = self.pool.urlopen(method, url, body, retries=False, headers=self.headers, **kw)
  File "C:\Users\BigData\Anaconda2\lib\site-packages\urllib3\connectionpool.py", line 609, in urlopen
    _stacktrace=sys.exc_info()[2])
  File "C:\Users\BigData\Anaconda2\lib\site-packages\urllib3\util\retry.py", line 222, in increment
    raise six.reraise(type(error), error, _stacktrace)
  File "C:\Users\BigData\Anaconda2\lib\site-packages\urllib3\connectionpool.py", line 559, in urlopen
    body=body, headers=headers)
  File "C:\Users\BigData\Anaconda2\lib\site-packages\urllib3\connectionpool.py", line 353, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "C:\Users\BigData\Anaconda2\lib\httplib.py", line 1053, in request
    self._send_request(method, url, body, headers)
  File "C:\Users\BigData\Anaconda2\lib\httplib.py", li

KeyboardInterrupt: 

In [54]:
# for i in doc:
#     print doc[i].decode('utf-8')

In [18]:
##step5 存進db
import pymongo
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017')
db = client['']  #選擇database
collect = db['']  #選擇database.collection

path_file = open('E:/patent_data/path_list_42.txt','r')
count = 0
for line in path_file.readlines():
    path_and_dim = line.strip()
    dic = read_file(path_and_dim)
    #print len(dic)
    try:
        collect.insert_one(dic)
        count += 1
    except pymongo.errors.DuplicateKeyError: #_id重複 (application-num)
        print '_id重複: ' + path_and_dim
        pass
    
    #print count,  dic['application-num'], path_and_dim
print count  #成功進筆數
path_file.close()

#可能出以下東西
#No spec:  抓不到specification, 可能原始檔有問題(亂碼)
#'_id重複:'

_id重複: E:/patent_data/42/PatentPubXML_042002/4202D165450/102306072-A0305.xml,D
_id重複: E:/patent_data/42/PatentPubXML_042019/4219I490473/101100273-A0305.xml,I
57623


特殊的
utility-model-specification
E:/patent_data/PatentPubXML_040011/4011M450881/101224447-A0305.xml,M